# Transfer Learning using Keras and ResNet-50 by IS - First Try
* 데이터셋은 64종류의 과일로 이루어진 42,345장의 이미지로 이루어져 있다. 
* 일반적인 접근방법과 전이 학습으로의 접근방법을 비교해볼 것이다.
* 2 epoch로 98.44%의 정확도를 얻을 수 있다.

**목차**는 다음과 같다:
1. 전이 학습(Transfer Learning)에 대한 간단한 설명
2. Kaggle Kernel을 이용한 전이 학습
3. 데이터를 로드하고 시각호
4. 모델 설계 및 컴파일
5. 미리 학습된 모델을 학습시키고 검증
6. 바닐라 모델(내가 순수 만든 모델)을 학습시키고 검증
7. 미리 학습된 모델과 바닐라모델과의 비교

In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

In [3]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

## 1. Transfer Learning
전이 학습에서 우리는 먼저 기본 데이터와 작업에 기반을 한 기본 신경망을 학습시키고 학습된 피처의 목적을 변경하거나 대상 데이터셋과 작업에 대한 학습을 할 두 번째 대상 신경망으로 전달한다. 이 과정은 피처가 기본 작업에 특정되지 않고 기본 작업과 대상 작업 모두에 적합한 일반적인 피처라면 잘 작동하는 경향이 있다.

Lisa Torrey와 Jude Shavlik은 전이 학습에 대한 챕터에서 전이학습을 사용할 때의 3가지 이점을 설명하고 있다:
* 더 빠른 출발이 가능하다 : 소스 모델의 초기 스킬이(모델을 정제하기 전) 그렇지 않은 경우보다 높다.
* 더 높은 경사 : 소스 모델을 학습시키는 동안 스킬의 향상속도가 그렇지 않은 경우보다 더 깊다.
* 더 높은 점근선(asymptote) : 학습된 모델의 수렴 스킬이 그렇지 않은 모델보다 더 좋다.

![transfer-learning-benefits](../img/transfer-learning-benefits.png)

기본적으로 미리 학습된 모델(다른 누군가에 의해 매우 큰 데이터셋으로 학습이 완료된 신경망의 가중치와 파라미터들)을 가져와서 우리 자신의 데이터셋으로 "세밀 조정(fine tuning)"을 한다. 전이 학습의 아이디어는 미리 학습된 모델은 초기화된 가중치를 제공하여 수렴 속도를 높이거나 관심 업무를 위한 고정 피처추출기 역할을 하는 것이다.

두 가지 주요 전이학습 시나리오는 다음과 같다:
* ConvNet 세밀 조정(Fine Tuning)하기: 임의의 값으로 가중치를 초기화시키는 대신 미리 학습된 신경망의 가중치로 신경망을 초기화한다. 예를 들면 ImageNet 1000과 같은 큰 데이터셋으로 미리 훈련된 신경망 말이다. 이 시나리오에서는 전체적인 신경망이 우리과 관심있는 데이터셋으로 다시 학습되어야 할 필요가 있다.
* ConvNet을 고정 피처 추출기로 사용하기 : 여기서 최종 완결연결계층을 제외한 모든 신경망에 대한 가중치를 동결(freeze)한다. (멈춘다의 의미인가?) 이 마지막 완전연결 계층은 임의의 가중치로 이루어진 새로운 층으로 대체되고 이 층만 학습시킨다.

이 노트북에서는 첫 번째 시나리오를 따른다.

## 2. Transfer Learning using Kaggle Kernels
### 2.1. Using the Keras Pretrained Models dataset
캐글 커널은 네트워크 연결이 되지 않으므로 미리 훈련된 케라스 모델을 다운로드 받을 수 없다. (주피터에서는 가능) [이 데이터셋](https://www.kaggle.com/moltean/fruits)은 우리가 원하는 미리 학습된 모델을 캐글 커널 환경에서 사용할 수 있도록 도와준다.

우리가 해줘야 할 것은 케라스가 찾고 있는 캐시 디렉토리(~/.keras/models)에 미리 학습된 모델을 복사하는 것이다.

In [4]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [6]:
!copy ../input/keras-pretrained-models/*notop* ~/.keras/models/
!copy ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
!copy ../input/keras-pretrained-models/resnet50* ~/.keras/models/

명령 구문이 올바르지 않습니다.
명령 구문이 올바르지 않습니다.
명령 구문이 올바르지 않습니다.
